In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bertbase/bert-base-uncased-vocab.txt
/kaggle/input/recsys/suggest_dump.txt


In [6]:
import torch.nn as nn
import torch

In [7]:
a = torch.randn((200000,100)).cuda()

In [8]:
b = []
cos=nn.CosineSimilarity(dim=0).cuda()
i=a[1]
c=[]
for j in tqdm.tqdm(a):
    c.append(cos(i.view(-1,1),j.view(-1,1)).tolist())
b.append(c)    

NameError: name 'tqdm' is not defined

In [9]:
torch.topk(torch.tensor(b).view(1,-1).cuda(),k=10).indices.tolist()

RuntimeError: invalid argument 5: k not in range for dimension at /opt/conda/conda-bld/pytorch_1591914880026/work/aten/src/THC/generic/THCTensorTopK.cu:23

In [10]:
train=pd.read_csv('../input/recsys/suggest_dump.txt',delimiter='\t')

In [11]:
train['id']=1

In [12]:
import shortuuid
import tqdm

In [13]:
train.loc[1,'id']

1

In [14]:
for i in tqdm.tqdm(range(len(train))):
    train.loc[i,'id']=shortuuid.uuid()

100%|██████████| 197465/197465 [00:37<00:00, 5214.80it/s]


In [15]:
train.head()

,authors,publisher,link,title,summary,subjects,tasks,year,id
0,"Jagadeesan Jayender,Haoyin Zhou",arxiv,https://arxiv.org/abs/2007.08553v1,Smooth Deformation Field-based Mismatch Remova...,Abstract: This paper studies the mismat...,"Computer Science,Computer Vision And Pattern R...",NaN,2020-20-20,SNqRRH9FnjbcvfLse3DpfJ
1,"Xu Chen,Lin Chen,Xin Tang,Shuai Yu,Liekang Zeng",arxiv,https://arxiv.org/abs/2007.09072v1,Joint Multi-User DNN Partitioning and Computat...,Abstract: Mobile Edge Computing (MEC) h...,"Computer Science,Distributed, Parallel, And Cl...",NaN,2020-20-20,Yd5KdsYkSPTpi6ycCoctPE
2,"Huchuan Lu,Lihe Zhang,Youwei Pang,Xiaoqi Zhao",arxiv,https://arxiv.org/abs/2007.09062v1,Multi-scale Interactive Network for Salient Ob...,Abstract: Deep-learning based salient o...,"Computer Science,Computer Vision And Pattern R...","Object Detection,Salient Object Detection",2020-20-20,K5vxRZ7rkVSyzrLymDgc2V
3,"Parth Patel,Kalpdrum Passi,Chakresh Kumar Jain",arxiv,https://arxiv.org/abs/2007.08652v1,Prediction of Cancer Microarray and DNA Methyl...,"Abstract: Over the past few years, ther...","Quantitative Biology,Quantitative Methods",NaN,2020-20-20,MJuwaV7GX83Qcfn4Dhkk6E
4,"Danail Stoyanov,Emanuele Colleoni,Philip Edwards",arxiv,https://arxiv.org/abs/2007.09107v1,Synthetic and Real Inputs for ToolSegmentation...,Abstract: Semantic tool segmentation in...,"Computer Science,Computer Vision And Pattern R...",Scene Understanding,2020-20-20,axkLK5TUkMEvwxWEJQhWBw


In [16]:
corpus = train['authors'].fillna('')+' '+train['title'].fillna('')+' '+train['summary'].fillna('')+' '+train['subjects'].fillna('')

In [17]:
corpus.index = train['id']
corpus.name = 'text'

In [18]:
for i in tqdm.tqdm(range(len(corpus)//10000+1)):
    corpus[10000*i:10000*(i+1)].to_csv('corpus{}.csv'.format(i))

100%|██████████| 20/20 [00:10<00:00,  2.00it/s]


In [19]:
from torchtext.data import Field,RawField, TabularDataset

In [20]:
import multiprocessing
import os
import re
import signal
from math import ceil
from os.path import join
from time import time
import numpy as np
import torch
from numpy.random import choice
from torchtext.data import RawField,Field, TabularDataset
from spacy.lang.en import STOP_WORDS
import string

class DataMaker():
    def __init__(self, datapath, batch_size, context_size,
                 num_noise_words=20, max_size=0, num_workers=None):
        self.datapath=datapath
        self.batch_size = batch_size
        self.context_size = context_size
        self.num_noise_words = num_noise_words
        self.max_size = max_size
        self.num_workers = num_workers
        self.stopwords = list(STOP_WORDS)+list((''.join(string.punctuation)).strip(''))+['-pron-','-PRON-']
        self.dataset = None
        
    def _tokenize_str(self,str_):
        # keep only alphanumeric and punctations
        str_ = re.sub(r'[^A-Za-z0-9(),.!?\'`]', ' ', str_)
        # remove multiple whitespace characters
        str_ = re.sub(r'\s{2,}', ' ', str_)
        # punctations to tokens
        str_ = re.sub(r'\(', ' ( ', str_)
        str_ = re.sub(r'\)', ' ) ', str_)
        str_ = re.sub(r',', ' , ', str_)
        str_ = re.sub(r'\.', ' . ', str_)
        str_ = re.sub(r'!', ' ! ', str_)
        str_ = re.sub(r'\?', ' ? ', str_)
        # split contractions into multiple tokens
        str_ = re.sub(r'\'s', ' \'s', str_)
        str_ = re.sub(r'\'ve', ' \'ve', str_)
        str_ = re.sub(r'n\'t', ' n\'t', str_)
        str_ = re.sub(r'\'re', ' \'re', str_)
        str_ = re.sub(r'\'d', ' \'d', str_)
        str_ = re.sub(r'\'ll', ' \'ll', str_)
        # lower case
        return str_.strip().lower().split()
    
    def getdata(self):
        start_time=time()
        doc_field = RawField()
        text_field = Field(stop_words=self.stopwords,pad_token=None,tokenize=self._tokenize_str)
        dataset = TabularDataset(self.datapath,skip_header=True,format='CSV',fields=[('id',doc_field),('text',text_field)])
        text_field.build_vocab(dataset)
        self.dataset=dataset
        print('Time taken --- {} seconds'.format(time()-start_time))
        return dataset
    
    def getgenerator(self):
        return NCEData(self.dataset,self.batch_size,self.context_size,self.num_noise_words,self.max_size,self.num_workers)


class NCEData(object):
    
    # code inspired by parallel generators in https://github.com/fchollet/keras
    def __init__(self, dataset, batch_size, context_size,
                 num_noise_words, max_size, num_workers):
        self.max_size = max_size

        self.num_workers = num_workers if num_workers != -1 else os.cpu_count()
        if self.num_workers is None:
            self.num_workers = 1

        self._generator = _NCEGenerator(
            dataset,
            batch_size,
            context_size,
            num_noise_words,
            _NCEGeneratorState(context_size))

        self._queue = None
        self._stop_event = None
        self._processes = []

    def __len__(self):
        return len(self._generator)

    def vocabulary_size(self):
        return self._generator.vocabulary_size()

    def start(self):
        """Starts num_worker processes that generate batches of data."""
        self._queue = multiprocessing.Queue(maxsize=self.max_size)
        self._stop_event = multiprocessing.Event()

        for _ in range(self.num_workers):
            process = multiprocessing.Process(target=self._parallel_task)
            process.daemon = True
            self._processes.append(process)
            process.start()

    def _parallel_task(self):
        while not self._stop_event.is_set():
            try:
                batch = self._generator.next()
                # queue blocks a call to put() until a free slot is available
                self._queue.put(batch)
            except KeyboardInterrupt:
                self._stop_event.set()

    def get_generator(self):
        """Returns a generator that yields batches of data."""
        while self._is_running():
            yield self._queue.get()

    def stop(self):
        """Terminates all processes that were created with start()."""
        if self._is_running():
            self._stop_event.set()

        for process in self._processes:
            if process.is_alive():
                os.kill(process.pid, signal.SIGINT)
                process.join()

        if self._queue is not None:
            self._queue.close()

        self._queue = None
        self._stop_event = None
        self._processes = []

    def _is_running(self):
        return self._stop_event is not None and not self._stop_event.is_set()


class _NCEGenerator(object):
    """An infinite, process-safe batch generator for noise-contrastive
    estimation of word vector models.
    Parameters
    ----------
    state: paragraphvec.data._NCEGeneratorState
        Initial (indexing) state of the generator.
    For other parameters see the NCEData class.
    """
    def __init__(self, dataset, batch_size, context_size,
                 num_noise_words, state):
        self.dataset = dataset
        self.batch_size = batch_size
        self.context_size = context_size
        self.num_noise_words = num_noise_words

        self._vocabulary = self.dataset.fields['text'].vocab
        self._sample_noise = None
        self._init_noise_distribution()
        self._state = state

    def _init_noise_distribution(self):
        # we use a unigram distribution raised to the 3/4rd power,
        # as proposed by T. Mikolov et al. in Distributed Representations
        # of Words and Phrases and their Compositionality
        probs = np.zeros(len(self._vocabulary) - 1)

        for word, freq in self._vocabulary.freqs.items():
            probs[self._word_to_index(word)] = freq

        probs = np.power(probs, 0.75)
        probs /= np.sum(probs)

        self._sample_noise = lambda: choice(
            probs.shape[0], self.num_noise_words, p=probs).tolist()

    def __len__(self):
        num_examples = sum(self._num_examples_in_doc(d) for d in self.dataset)
        return ceil(num_examples / self.batch_size)

    def vocabulary_size(self):
        return len(self._vocabulary) - 1

    def next(self):
        """Updates state for the next process in a process-safe manner
        and generates the current batch."""
        prev_doc_id, prev_in_doc_pos = self._state.update_state(
            self.dataset,
            self.batch_size,
            self.context_size,
            self._num_examples_in_doc)

        # generate the actual batch
        batch = _NCEBatch(self.context_size)

        while len(batch) < self.batch_size:
            if prev_doc_id == len(self.dataset):
                # last document exhausted
                batch.torch_()
                return batch
            if prev_in_doc_pos <= (len(self.dataset[prev_doc_id].text) - 1
                                   - self.context_size):
                # more examples in the current document
                self._add_example_to_batch(prev_doc_id, prev_in_doc_pos, batch)
                prev_in_doc_pos += 1
            else:
                # go to the next document
                prev_doc_id += 1
                prev_in_doc_pos = self.context_size

        batch.torch_()
        return batch

    def _num_examples_in_doc(self, doc, in_doc_pos=None):
        if in_doc_pos is not None:
            # number of remaining
            if len(doc.text) - in_doc_pos >= self.context_size + 1:
                return len(doc.text) - in_doc_pos - self.context_size
            return 0

        if len(doc.text) >= 2 * self.context_size + 1:
            # total number
            return len(doc.text) - 2 * self.context_size
        return 0

    def _add_example_to_batch(self, doc_id, in_doc_pos, batch):
        doc = self.dataset[doc_id].text
        batch.doc_ids.append(doc_id)

        # sample from the noise distribution
        current_noise = self._sample_noise()
        current_noise.insert(0, self._word_to_index(doc[in_doc_pos]))
        batch.target_noise_ids.append(current_noise)

        if self.context_size == 0:
            return

        current_context = []
        context_indices = (in_doc_pos + diff for diff in
                           range(-self.context_size, self.context_size + 1)
                           if diff != 0)

        for i in context_indices:
            context_id = self._word_to_index(doc[i])
            current_context.append(context_id)
        batch.context_ids.append(current_context)

    def _word_to_index(self, word):
        return self._vocabulary.stoi[word] - 1


class _NCEGeneratorState(object):
    """Batch generator state that is represented with a document id and
    in-document position. It abstracts a process-safe indexing mechanism."""
    def __init__(self, context_size):
        # use raw values because both indices have
        # to manually be locked together
        self._doc_id = multiprocessing.RawValue('i', 0)
        self._in_doc_pos = multiprocessing.RawValue('i', context_size)
        self._lock = multiprocessing.Lock()

    def update_state(self, dataset, batch_size,
                     context_size, num_examples_in_doc):
        """Returns current indices and computes new indices for the
        next process."""
        with self._lock:
            doc_id = self._doc_id.value
            in_doc_pos = self._in_doc_pos.value
            self._advance_indices(
                dataset, batch_size, context_size, num_examples_in_doc)
            return doc_id, in_doc_pos

    def _advance_indices(self, dataset, batch_size,
                         context_size, num_examples_in_doc):
        num_examples = num_examples_in_doc(
            dataset[self._doc_id.value], self._in_doc_pos.value)

        if num_examples > batch_size:
            # more examples in the current document
            self._in_doc_pos.value += batch_size
            return

        if num_examples == batch_size:
            # just enough examples in the current document
            if self._doc_id.value < len(dataset) - 1:
                self._doc_id.value += 1
            else:
                self._doc_id.value = 0
            self._in_doc_pos.value = context_size
            return

        while num_examples < batch_size:
            if self._doc_id.value == len(dataset) - 1:
                # last document: reset indices
                self._doc_id.value = 0
                self._in_doc_pos.value = context_size
                return

            self._doc_id.value += 1
            num_examples += num_examples_in_doc(
                dataset[self._doc_id.value])

        self._in_doc_pos.value = (len(dataset[self._doc_id.value].text)
                                  - context_size
                                  - (num_examples - batch_size))


class _NCEBatch(object):
    def __init__(self, context_size):
        self.context_ids = [] if context_size > 0 else None
        self.doc_ids = []
        self.target_noise_ids = []

    def __len__(self):
        return len(self.doc_ids)

    def torch_(self):
        if self.context_ids is not None:
            self.context_ids = torch.LongTensor(self.context_ids)
        self.doc_ids = torch.LongTensor(self.doc_ids)
        self.target_noise_ids = torch.LongTensor(self.target_noise_ids)

    def cuda_(self):
        if self.context_ids is not None:
            self.context_ids = self.context_ids.cuda()
        self.doc_ids = self.doc_ids.cuda()
        self.target_noise_ids = self.target_noise_ids.cuda()

In [22]:
data = DataMaker('corpus1.csv',batch_size=256,context_size=3,num_workers=-1)

In [23]:
dataset=data.getdata()

Time taken --- 2.756767988204956 seconds


In [24]:
generator = data.getgenerator()

In [25]:
len(generator)

3636

In [26]:
class NegativeSampling(nn.Module):
    """Negative sampling loss as proposed by T. Mikolov et al. in Distributed
    Representations of Words and Phrases and their Compositionality.
    """
    def __init__(self):
        super(NegativeSampling, self).__init__()
        self._log_sigmoid = nn.LogSigmoid()

    def forward(self, scores):
        """Computes the value of the loss function.
        Parameters
        ----------
        scores: autograd.Variable of size (batch_size, num_noise_words + 1)
            Sparse unnormalized log probabilities. The first element in each
            row is the ground truth score (i.e. the target), other elements
            are scores of samples from the noise distribution.
        """
        k = scores.size()[1] - 1
        return -torch.sum(
            self._log_sigmoid(scores[:, 0])
            + torch.sum(self._log_sigmoid(-scores[:, 1:]), dim=1) / k
        ) / scores.size()[0]

In [27]:
import torch
import torch.nn as nn


class DM(nn.Module):
    """Distributed Memory version of Paragraph Vectors.
    Parameters
    ----------
    vec_dim: int
        Dimensionality of vectors to be learned (for paragraphs and words).
    num_docs: int
        Number of documents in a dataset.
    num_words: int
        Number of distinct words in a daset (i.e. vocabulary size).
    """
    def __init__(self, vec_dim, num_docs, num_words):
        super(DM, self).__init__()
        # paragraph matrix
        self._D = nn.Parameter(
            torch.randn(num_docs, vec_dim), requires_grad=True)
        # word matrix
        self._W = nn.Parameter(
            torch.randn(num_words, vec_dim), requires_grad=True)
        # output layer parameters
        self._O = nn.Parameter(
            torch.FloatTensor(vec_dim, num_words).zero_(), requires_grad=True)

    def forward(self, context_ids, doc_ids, target_noise_ids):
        """Sparse computation of scores (unnormalized log probabilities)
        that should be passed to the negative sampling loss.
        Parameters
        ----------
        context_ids: torch.Tensor of size (batch_size, num_context_words)
            Vocabulary indices of context words.
        doc_ids: torch.Tensor of size (batch_size,)
            Document indices of paragraphs.
        target_noise_ids: torch.Tensor of size (batch_size, num_noise_words + 1)
            Vocabulary indices of target and noise words. The first element in
            each row is the ground truth index (i.e. the target), other
            elements are indices of samples from the noise distribution.
        Returns
        -------
            autograd.Variable of size (batch_size, num_noise_words + 1)
        """
        # combine a paragraph vector with word vectors of
        # input (context) words
        x = torch.add(
            self._D[doc_ids, :], torch.sum(self._W[context_ids, :], dim=1))

        # sparse computation of scores (unnormalized log probabilities)
        # for negative sampling
        return torch.bmm(
            x.unsqueeze(1),
            self._O[:, target_noise_ids].permute(1, 0, 2)).squeeze()

    def get_paragraph_vector(self):
        return self._D.data.tolist()
    
    def fit(self,generator,epochs,num_batches):
        
        opt=torch.optim.Adam(self.parameters(),lr=0.0001)
        cost_func = NegativeSampling()
        if torch.cuda.is_available():            
            cost_func.cuda()

        generator.start()        
        
        batch_maker = generator.get_generator()
        for epoch in range(epochs):
            step = 0
            loss=[]
            for batch_i in tqdm.tqdm(range(num_batches)):
#                 step+=1
                batch = next(batch_maker)
                if torch.cuda.is_available():
                    batch.cuda_()
                
                x = self.forward(
                        batch.context_ids,
                        batch.doc_ids,
                        batch.target_noise_ids) 
                x = cost_func.forward(x)
                loss.append(x.item())
                self.zero_grad()
                x.backward()
                opt.step()
#                 if step%100==0:
#                     print('-',end='')
            loss = torch.mean(torch.FloatTensor(loss))
            print('epoch - {} loss - {:.4f}'.format(epoch+1,loss))
        generator.stop()
        
    def get_embeddings(self,ids):
        docvecs = self._D.data.tolist()
        if len(docvecs)!=len(ids):
            raise("Length of ids does'nt match")
            return 0
        self.embeddings = pd.DataFrame({'ids':ids,'vector':docvecs})
        return self.embeddings
    
    def most_similar_docs(self,docs,topk=11):
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        if not isinstance(docs,list):
            docs = [docs]
            
        if not self.embeddings.ids.isin(docs).any():
            raise('Not in vocab')
            return 0
        
        index = self.embeddings['vector'][self.embeddings['ids'].isin(docs)].values.tolist()
        index = torch.LongTensor(index).to(device)
        embeddings = torch.FloatTensor(self.embeddings['vector'].values.tolist()).to(device)
        similars = self._similarity(index,embeddings,topk)
        similar_docs = self.embeddings['ids'][similars.indices.tolist()[0]].values.tolist()
        probs = similars.values.tolist()[0]
        
        return similar_docs[1:], probs[1:]
        
    def _similarity(self,doc,embeddings,topk):
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        similarity = []
        
        cos=nn.CosineSimilarity(dim=0).to(device)
        for i in doc:
            inner = []
            for j in embeddings:
                inner.append(cos(i.view(-1,1),j.view(-1,1)).tolist())
            similarity.append(inner)
        similarity = torch.FloatTensor(similarity).view(1,-1).to(device)
        return torch.topk(similarity,topk)

In [28]:
model = DM(vec_dim=100,num_docs=len(dataset),num_words=generator.vocabulary_size()).cuda()

In [29]:
generator.vocabulary_size()

107701

In [30]:
len(generator)

3636

In [31]:
model.fit(generator,num_batches=len(generator),epochs=5)

  0%|          | 0/3636 [00:00<?, ?it/s]

epoch - 1 loss - 1.3200


  0%|          | 0/3636 [00:00<?, ?it/s]

epoch - 2 loss - 1.1657


 23%|██▎       | 830/3636 [02:45<09:19,  5.02it/s]


KeyboardInterrupt: 

In [32]:
getids = pd.read_csv('corpus1.csv')

In [33]:
getids.id

0       282JrhE8B9dJN8sJF3TEzE
1       Ff3j8fApeY7iFavuCDfsKS
2       7cNmGgcR2eZ2gB6yB3kh7r
3       QkQvYBXXXtKWhtAQNf8UFn
4       fmherzLQ7FYVUCvrpeLBbT
                 ...          
9995    3xNSwsDFzA7smPvzhzFVVQ
9996    HLHECNEA8i4u6TAdCSjYaZ
9997    YBr8yMHuew7VYvxYYNSgmr
9998    JGC6izRWCA8rwfTaffbFvt
9999    UmKbFbBhcGy2q7u92BrYJy
Name: id, Length: 10000, dtype: object

In [34]:
e = model.get_embeddings(getids['id'].values.tolist())

In [36]:
sim = model.most_similar_docs(['YBr8yMHuew7VYvxYYNSgmr'])

In [166]:
train[train['id'].isin(sim[0]+['YBr8yMHuew7VYvxYYNSgmr'])]

,authors,publisher,link,title,summary,subjects,tasks,year,id
1032,"Xu Wang,Mladen Kolar,Ali Shojaie",arxiv,https://arxiv.org/abs/2007.07448v1,Statistical Inference for Networks of High-Dim...,Fueled in part by recent applications ...,"Machine Learning,Statistics",Point Processes,2020-20-20,gMAoeMzckx75dFpRexijWa
1060,"Cheng Wang,Zhou Tang,Zhangsheng Yu",arxiv,https://arxiv.org/abs/1901.07150,A Fast Iterative Algorithm for High-dimensiona...,Differential network is an important tool to c...,"Statistics,Computation",NaN,2019-19-19,Uvq2FjtCnnQuL39hAt4whM
1122,"Won-Ki Jeong,Tran Minh Quan,Tuan Tran Anh,Khoa...",arxiv,https://arxiv.org/abs/2005.07058,Reinforced Coloring for End-to-End Instance Se...,Instance segmentation is one of the actively s...,"Computer Science,Computer Vision And Pattern R...",NaN,2020-20-20,2xWqvsNBjphZa6wKPbpU2S
1201,"Anima Anandkumar,Wuyang Chen,Zhangyang Wang,Zh...",arxiv,https://arxiv.org/abs/2007.06965,Automated Synthetic-to-Real Generalization,Models trained on synthetic images often face ...,"Computer Science,Machine Learning",NaN,2020-20-20,k3QFPSXqXUdSxJGoJKt2VY
1216,"Xiang Yue,Feng Huang,Wen Zhang,Zhankun Xiong,Z...",arxiv,https://arxiv.org/abs/1911.05584,Tensor Decomposition with Relational Constrain...,MicroRNAs (miRNAs) play crucial roles in multi...,"Computer Science,Machine Learning",NaN,2019-19-19,jExxTwPuvNQQkv75Bn5MaC
1325,"Matthias Schubert,Michael Shekelyan,Gregor Jossé",arxiv,https://arxiv.org/abs/1410.0205,ParetoPrep: Fast computation of Path Skylines ...,Computing cost optimal paths in network data i...,"Computer Science,Databases",NaN,2014-14-14,kBjkemWn6XAAGoojSb82HT
1527,"Dongming Yang,YueXian Zou",arxiv,https://arxiv.org/abs/2007.06925v1,A Graph-based Interactive Reasoning for Human-...,Human-Object Interaction (HOI) detecti...,"Computer Science,Computer Vision And Pattern R...",Human-Object Interaction Detection,2020-20-20,64J4DhtLCB5rExACY8uJtV
1811,"Lucas Gabriel Souza França,Pedro Montoya,José ...",arxiv,https://arxiv.org/abs/1702.03912,On multifractals: a non-linear study of actigr...,"This work aimed, to determine the characterist...","Nonlinear Sciences,Chaotic Dynamics",NaN,2017-17-17,FGEtG7cJozuP7fVn4hLfuV
1971,"Christopher De Sa,Karen Levy,A. Feder Cooper",arxiv,https://arxiv.org/abs/2007.02203,Regulating Accuracy-Efficiency Trade-Offs in D...,In this paper we discuss the trade-off between...,"Computer Science,Computers And Society",NaN,2020-20-20,8WjtQprVejGDZB3Asay2We
1988,"Clayton G. Webster,Joseph Daws Jr.",arxiv,https://arxiv.org/abs/1905.10457,A Polynomial-Based Approach for Architectural ...,In this effort we propose a novel approach for...,"Computer Science,Machine Learning",NaN,2019-19-19,LspcZfnPuqpmf2UpE5bskA
